In [3]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
src_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_dir not in sys.path:
    sys.path.append(src_dir)
from modules.preprocess import *
from modules.config_loader import load_config

In [4]:
maker_df=pd.read_csv("../../data/250611_SUN_MAKER_STD.csv")
model_df=pd.read_csv("../../data/250611_SUN_MODEL_STD.csv")
tracker_df=pd.read_csv("../../data/250611_sun_tracker_Data.csv")

/tmp/ipykernel_754939/465895119.py:3: DtypeWarning: Columns (11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  tracker_df=pd.read_csv("../../data/250611_sun_tracker_Data.csv")


In [5]:
config = load_config()
EQUIPMENTS_CATEGORY = config["EQUIPMENTS_CATEGORY_CODE"]
PARTS_CATEGORY = config["PARTS_CATEGORY_CODE"]
OTHERS_CATEGORY = config["OTHERS_CATEGORY_CODE"]

In [6]:
tracker_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897052 entries, 0 to 897051
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   EQUIP_ID              897052 non-null  object
 1   EQUIP_TX_CNT          897052 non-null  int64 
 2   Category_Code         897052 non-null  object
 3   Category_Name         897052 non-null  object
 4   MAKER_ID              897052 non-null  object
 5   MAKER_NAME            896486 non-null  object
 6   MODEL_ID              897052 non-null  object
 7   MODEL_NAME            896796 non-null  object
 8   Process_NM            390359 non-null  object
 9   Configuration         632622 non-null  object
 10  SERIAL_NO             157417 non-null  object
 11  VINTAGE               102921 non-null  object
 12  Tracker_availability  897052 non-null  object
 13  Seller                875796 non-null  object
 14  Owner                 829420 non-null  object
 15  USER_FAB_ID      

# standard model에서 장비만 추출

In [7]:
equipment_model_df=model_df[model_df['categoryCD'].isin(EQUIPMENTS_CATEGORY)]
parts_model_df=model_df[model_df['categoryCD'].isin(PARTS_CATEGORY)]
others_model_df=model_df[model_df['categoryCD'].isin(OTHERS_CATEGORY)]

In [8]:
equipment_model_df

,modelId,modelSTDName,makerId,modelAliasList,categoryCD
64,MO15169,Optima MD,MA225,Optima MD,IMPLANT
65,MO68,300XP,MA2456,"300XP, 300XPM IMPLANTER - 300XP #2, ION IMPLA...",IMPLANT
66,MO1733,NV-8250,MA225,"8250, NV8250, NV-8250",IMPLANT
3518,MO358264,Purion H200,MA225,Purion H200,IMPLANT
3519,MO1746,NV-GSDIII-180,MA1997,"GSD III - 180, GSD180, GSD3 180, GSD-3-180, GS...",IMPLANT
...,...,...,...,...,...
398575,MO520,CM402-L (A),MA1691,"CM402A, CM402Atype, CM402-L (A type), CM402-L ...",CHIP_MOUNTER
398577,MO5904,SC-410,MA2490,SC-410,PKG_ETC
398578,MO5933,YS24 Dual,MA2574,YS24 Dual,CHIP_MOUNTER
398579,MO5934,YS24 Single,MA2574,YS24 Single,CHIP_MOUNTER


In [9]:
others_model_df = others_model_df[others_model_df["modelSTDName"] != "unknown"]
equipment_model_df = equipment_model_df[equipment_model_df["modelSTDName"] != "unknown"]
result = pd.concat([equipment_model_df, others_model_df], ignore_index=True)

In [11]:
tracker_df["Process_NM"] = tracker_df["Process_NM"].fillna(tracker_df["Process(model Info)"])
tracker_df["Configuration"] = tracker_df["Configuration"].fillna(tracker_df["config(model Info)"])
tracker_df["Process_NM"] = tracker_df["Process_NM"].apply(lambda x: str(x)[:20] if pd.notna(x) else x)
tracker_df["Configuration"] = tracker_df["Configuration"].apply(lambda x: str(x)[:20] if pd.notna(x) else x)


In [ ]:
tracker_df.drop(["EQUIP_ID","EQUIP_TX_CNT","Category_Code","USER_FAB_ID",
                   "COUNTRY_CD","CODE","Process(model Info)","config(model Info)"], axis=1, inplace=True)
tracker_df = tracker_df.dropna(subset=["MODEL_NAME"])

,EQUIP_ID,EQUIP_TX_CNT,Category_Code,Category_Name,MAKER_ID,MAKER_NAME,MODEL_ID,MODEL_NAME,Process_NM,Configuration,...,Seller,Owner,USER_FAB_ID,COUNTRY_CD,P&C,CODE,STD_MODEL_YN,STD_MAKER_YN,Process(model Info),config(model Info)
0,E1,1,GENERAL_TESTER,General Tester,MA45,Advantest,MO6491,8595E,Spectrum Analyzer,NaN,...,Index Solution,"DOULTECH Co., Ltd",NaN,NaN,N,NaN,Y,Y,NaN,NaN
1,E10,1,REFLOW_SOLDERING,Reflow&Soldering,MA752,Furukawa,MO6845,XNIII-725PCG,Reflow oven_,NaN,...,JINUTEC CORPORATION,JINUTEC CORPORATION,NaN,NaN,N,NaN,Y,Y,NaN,NaN
2,E100,1,PRINTER_DISPENSER,Printer&Dispenser,MA881,Hitachi,MO2604,TPM-1100,Screen Printer,NaN,...,"Kunshan Wospring Electronics Technology Co.,Ltd","Kunshan Wospring Electronics Technology Co.,Ltd",NaN,KR,N,NaN,Y,Y,NaN,NaN
3,E1000,1,CHIP_MOUNTER,Chip Mounter,MA740,Fuji,MO588,CP-842E,Chip mounter,NaN,...,DHC,DHC,NaN,NaN,N,NaN,Y,Y,NaN,NaN
4,E10000,1,DICING_SAW,Dicing Saw,MA522,Disco,MO719,DFD640,Wafer Saw,Including CSP Functi,...,SEMI RUN,SEMI RUN,NaN,KR,N,NaN,Y,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897047,E999995,1,PARTS_PC,Part Others,MA110,AMAT,MO400396,0150-20124,5500 PVD MAINFRAME,Qty : 2 / Title : Ap,...,"Capitol Area Technology, LLC.","Capitol Area Technology, LLC.",NaN,US,Y,NaN,Y,Y,5500 PVD MAINFRAME,"Desc.: CABLE ASSY, GROUND STRAP ,LID, Consumab..."
897048,E999996,1,PARTS_PC,Part Others,MA262092,TEL,MO659810,ES3D05-200162-12,Etch system,Qty : 5 / Title : TE,...,"Capitol Area Technology, LLC.","Capitol Area Technology, LLC.",NaN,US,Y,NaN,Y,Y,Etch system,"Desc.: RING,SHIELD(G1)"
897049,E999997,1,PARTS_PC,Part Others,MA110,AMAT,MO400983,0200-35051,5200 ASD EPI,Qty : 3 / Title : Ap,...,"Capitol Area Technology, LLC.","Capitol Area Technology, LLC.",NaN,US,Y,NaN,Y,Y,5200 ASD EPI,"Desc.: BAFFLE, INJECT NARROW,, Consumable: C"
897050,E999998,1,PARTS_PC,Part Others,MA283519,PARKER HANNIFIN CORP-COMPUMOTOR,MO797121,SVC2195M-7,NaN,Qty : 6 / Title : PA,...,"Capitol Area Technology, LLC.","Capitol Area Technology, LLC.",NaN,US,Y,NaN,N,N,NaN,NaN


In [16]:
equipment_tracker_df=tracker_df[tracker_df['Category_Code'].isin(EQUIPMENTS_CATEGORY)]
parts_tracker_df=tracker_df[tracker_df['Category_Code'].isin(PARTS_CATEGORY)]
others_tracker_df=tracker_df[tracker_df['Category_Code'].isin(OTHERS_CATEGORY)]

In [17]:
others_tracker_df=others_tracker_df[(others_tracker_df["STD_MODEL_YN"]=="Y") & (others_tracker_df["P&C"]=="N")]
equipment_tracker_df=equipment_tracker_df[(equipment_tracker_df["STD_MODEL_YN"]=="Y") & (equipment_tracker_df["P&C"]=="N")]
equipment_tracker_df=pd.concat([equipment_tracker_df, others_tracker_df], ignore_index=True)
equipment_tracker_df

,EQUIP_ID,EQUIP_TX_CNT,Category_Code,Category_Name,MAKER_ID,MAKER_NAME,MODEL_ID,MODEL_NAME,Process_NM,Configuration,...,Seller,Owner,USER_FAB_ID,COUNTRY_CD,P&C,CODE,STD_MODEL_YN,STD_MAKER_YN,Process(model Info),config(model Info)
0,E10,1,REFLOW_SOLDERING,Reflow&Soldering,MA752,Furukawa,MO6845,XNIII-725PCG,Reflow oven_,NaN,...,JINUTEC CORPORATION,JINUTEC CORPORATION,NaN,NaN,N,NaN,Y,Y,NaN,NaN
1,E1000,1,CHIP_MOUNTER,Chip Mounter,MA740,Fuji,MO588,CP-842E,Chip mounter,NaN,...,DHC,DHC,NaN,NaN,N,NaN,Y,Y,NaN,NaN
2,E10000,1,DICING_SAW,Dicing Saw,MA522,Disco,MO719,DFD640,Wafer Saw,Including CSP Functi,...,SEMI RUN,SEMI RUN,NaN,KR,N,NaN,Y,Y,NaN,NaN
3,E10001,1,WIRE_BONDER,Wire Bonder,MA179,ASM,MO430,Eagle60,Wire Bonder,NaN,...,"ENERCORE CO., LTD.","ENERCORE CO., LTD.",NaN,KR,N,NaN,Y,Y,Wire Bonding,NaN
4,E10002,1,WIRE_BONDER,Wire Bonder,MA179,ASM,MO430,Eagle60,Wire Bonder,NaN,...,"ENERCORE CO., LTD.","ENERCORE CO., LTD.",NaN,KR,N,NaN,Y,Y,Wire Bonding,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217302,E992781,1,MICROSCOPE,Microscope,MA1639,Olympus,MO236947,STM-UM,Kenbikyou (Sokubikyo,NaN,...,Nuvoton Technology Corporation Japan,Nuvoton Technology Corporation Japan,NaN,JP,N,NaN,Y,Y,NaN,NaN
217303,E992784,1,MICROSCOPE,Microscope,MA1561,Nikon,MO65038,SMZ-645,Stereo microscope,NaN,...,Nuvoton Technology Corporation Japan,Nuvoton Technology Corporation Japan,NaN,JP,N,NaN,Y,Y,NaN,NaN
217304,E992787,1,MICROSCOPE,Microscope,MA1561,Nikon,MO65038,SMZ-645,Stereo microscope,NaN,...,Nuvoton Technology Corporation Japan,Nuvoton Technology Corporation Japan,NaN,JP,N,NaN,Y,Y,NaN,NaN
217305,E992792,1,MICROSCOPE,Microscope,MA1561,Nikon,MO65038,SMZ-645,Stereo microscope,NaN,...,Nuvoton Technology Corporation Japan,Nuvoton Technology Corporation Japan,NaN,JP,N,NaN,Y,Y,NaN,NaN


In [20]:
result_tracker = equipment_tracker_df.groupby("MODEL_ID").agg(
    lambda x: ', '.join(set(map(str, x.dropna())))
).reset_index()
result_tracker=result_tracker[result_tracker["MODEL_NAME"]!="Unknown"]

result_tracker

,MODEL_ID,EQUIP_ID,EQUIP_TX_CNT,Category_Code,Category_Name,MAKER_ID,MAKER_NAME,MODEL_NAME,Process_NM,Configuration,...,Seller,Owner,USER_FAB_ID,COUNTRY_CD,P&C,CODE,STD_MODEL_YN,STD_MAKER_YN,Process(model Info),config(model Info)
1,MO1,"E1005686, E697314, E435618, E1005677, E532657,...",1,DIE_BONDER,Die Bonder,MA644,ESEC,2008 HS3,"Die Bonding System , Die bonding, Automatic pl...",weight : 900kg,...,"Dynamic Mos Technology Co.,Ltd., JCET Group Co...","Micron Semiconductor Malaysia Sdn. Bhd., Dynam...",,"TW, CN, MY",N,"ES28HS3-0002, 40002598, 40002113, 40001956, 24...",Y,Y,,
2,MO100,"E475471, E741799, E412994, E475516, E475521, E...",1,GENERAL_TESTER,General Tester,MA10283,Agilent,4156A,"Inspection / Measure, Parameter Analyzer, PARA...","전략물자유무 (ECCN Code) :, , TESTR-SO",...,"SurplusGLOBAL, Inc., Sumitomo Mitsui Finance a...","TEKmart International, Inc., Freescale Kilbrid...",,"US, JP, CN, KR",N,"534HH-05800, 16240, C99714100767, 199514300711...",Y,Y,,
3,MO1000,"E405792, E18293, E18295, E518779, E18294, E151...",1,STEPPER,Stepper,MA321,Canon,FPA-1550M4W,"Steppers, g-line, 1/5G LINE STEPPER, Wafer Ste...",", Remark : 5% , Tool Condition : Wit, Re...",...,"Evatec AG, CSMC Technologies Corporation, BOT ...","CSMC Technologies Corporation, BOT Lease Co.,L...",,JP,N,"FPA50, D0525, D0522, S00054231, FPA58, 100304,...",Y,Y,,
4,MO10003,"E649515, E25396",1,STEPPER,Stepper,MA2052,Seiko Instrument Inc.,SMI8300,"FIB, scanning",,...,"Sumitomo Mitsui Finance and Leasing Co., Ltd.,...","ROHM Tsukuba Co.,Ltd",,,N,"CIFB01, 25469",Y,Y,scanning,
5,MO10004,"E649516, E649517, E21270, E26418, E234435, E74...",1,METROLOGY,Metrology,MA2052,Seiko Instrument Inc.,SPA500,"AFM, INSPECTION","SCANNING ATOMIC FORC, ASSET No : 44000193-,",...,"ATREG, Inc, Samsung Electronics Co., Ltd., ROH...","Sumitomo Mitsui Finance and Leasing Co., Ltd.,...",,JP,N,"QAF1, 176ＱＡＦ１, 20070606F4m01, 6052, 177ＱＡＦ１, R...",Y,Y,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14928,MO9950,"E350754, E350756, E350751, E22507, E350755, E4...",1,WIRE_BONDER,Wire Bonder,MA2035,Shinkawa,UTC5100,"Bonder, Wire Bonder",Wire Bonder,...,"Toshiba Corporation, Summit Equipment Disposio...","Toshiba Corporation, Summit Equipment Disposio...",,,N,"WB623, WB627, 51383, WB626, WB625, WB628, WB624",Y,Y,,
14929,MO9960,"E21908, E53824",1,METROLOGY,Metrology,MA2039,Shinsung ENG,SGL-30,QE,,...,"SKhynix Semiconductor (China) Ltd, GEF","SKhynix Semiconductor (China) Ltd, GEF",,KR,N,FPVS01,Y,Y,,
14930,MO997,"E25485, E694025, E240323, E672853, E672854, E6...",1,STEPPER,Stepper,MA321,Canon,FPA-1550M2,"g-Line Stepper , PHOTO(Stepper), g-Line Steppe...",Stepper,...,"The Innes Group, Sumitomo Mitsui Finance and L...","ROHM Tsukuba Co.,Ltd, KEC Corporation, AMI Sem...",,KR,N,"(M-153), M-189, M-151, M-153, 90100, (M-146), ...",Y,Y,,
14931,MO998,"E240324, E474699, E649511, E18296, E18298, E69...",1,STEPPER,Stepper,MA321,Canon,FPA-1550M3-W,"STEPPER Main Body/Fi, PHOTO G-LINE STEPPER, PH...","RETICLE SIZE：φ125ｍｍ, STEPPER Main Body/Fi, A/E",...,"Renesas Electronics Corporation, KEC Corporati...","Hangzhou E-Giant Semiconductor Tech Co., LTD, ...",,,N,"FPA43, E-ST-0092, FPA46, PH12A, M-148, FPA44, ...",Y,Y,,


In [21]:
result_tracker.drop(["EQUIP_ID","EQUIP_TX_CNT","Category_Code","USER_FAB_ID",
                   "COUNTRY_CD","CODE","Process(model Info)","config(model Info)"], axis=1, inplace=True)
result_tracker = result_tracker.dropna(subset=["MODEL_NAME"])

In [23]:
result

,modelId,modelSTDName,makerId,modelAliasList,categoryCD
0,MO15169,Optima MD,MA225,Optima MD,IMPLANT
1,MO68,300XP,MA2456,"300XP, 300XPM IMPLANTER - 300XP #2, ION IMPLA...",IMPLANT
2,MO1733,NV-8250,MA225,"8250, NV8250, NV-8250",IMPLANT
3,MO358264,Purion H200,MA225,Purion H200,IMPLANT
4,MO1746,NV-GSDIII-180,MA1997,"GSD III - 180, GSD180, GSD3 180, GSD-3-180, GS...",IMPLANT
...,...,...,...,...,...
19338,MO1824,SP22P-M,MA1691,"SP22PM, SP-22P-M, SP22P-M",PRINTER_DISPENSER
19339,MO1844,SPF,MA1691,SPF,PRINTER_DISPENSER
19340,MO3028,YVP-XG,MA2574,"YVPXG, YVP-XG",PRINTER_DISPENSER
19341,MO5210,INTERA 1.5T,MA1721,INTERA 1.5T,MEDICAL


In [22]:
result_tracker

,MODEL_ID,Category_Name,MAKER_ID,MAKER_NAME,MODEL_NAME,Process_NM,Configuration,SERIAL_NO,VINTAGE,Tracker_availability,Seller,Owner,P&C,STD_MODEL_YN,STD_MAKER_YN
1,MO1,Die Bonder,MA644,ESEC,2008 HS3,"Die Bonding System , Die bonding, Automatic pl...",weight : 900kg,"207586, 207574, 207587, 207575, 207576",,"AVAIL, NOT_AVAIL, UNSPEC","Dynamic Mos Technology Co.,Ltd., JCET Group Co...","Micron Semiconductor Malaysia Sdn. Bhd., Dynam...",N,Y,Y
2,MO100,General Tester,MA10283,Agilent,4156A,"Inspection / Measure, Parameter Analyzer, PARA...","전략물자유무 (ECCN Code) :, , TESTR-SO","JP10A01132, JP10A00786, JP10A00265, JP10A50171...","1994.0, 1998.0, 1996.0","AVAIL, NOT_AVAIL, UNSPEC","SurplusGLOBAL, Inc., Sumitomo Mitsui Finance a...","TEKmart International, Inc., Freescale Kilbrid...",N,Y,Y
3,MO1000,Stepper,MA321,Canon,FPA-1550M4W,"Steppers, g-line, 1/5G LINE STEPPER, Wafer Ste...",", Remark : 5% , Tool Condition : Wit, Re...","11203, 507303M4, 6164, 7166, 7168, 200009, 515...","2000, 1991, 1995.0, 1991.0, 1999, 1992.0, 1992...","AVAIL, NOT_AVAIL, UNSPEC","Evatec AG, CSMC Technologies Corporation, BOT ...","CSMC Technologies Corporation, BOT Lease Co.,L...",N,Y,Y
4,MO10003,Stepper,MA2052,Seiko Instrument Inc.,SMI8300,"FIB, scanning",,,1990,UNSPEC,"Sumitomo Mitsui Finance and Leasing Co., Ltd.,...","ROHM Tsukuba Co.,Ltd",N,Y,Y
5,MO10004,Metrology,MA2052,Seiko Instrument Inc.,SPA500,"AFM, INSPECTION","SCANNING ATOMIC FORC, ASSET No : 44000193-,",#10200187000065,"2003.0, 2004.0","AVAIL, UNSPEC","ATREG, Inc, Samsung Electronics Co., Ltd., ROH...","Sumitomo Mitsui Finance and Leasing Co., Ltd.,...",N,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14928,MO9950,Wire Bonder,MA2035,Shinkawa,UTC5100,"Bonder, Wire Bonder",Wire Bonder,"A1-70415, A1-70417, A1-70419, A1-70420, A1-704...",,"NOT_AVAIL, UNSPEC","Toshiba Corporation, Summit Equipment Disposio...","Toshiba Corporation, Summit Equipment Disposio...",N,Y,Y
14929,MO9960,Metrology,MA2039,Shinsung ENG,SGL-30,QE,,"1110006, 11-10-005",2011.0,"AVAIL, UNSPEC","SKhynix Semiconductor (China) Ltd, GEF","SKhynix Semiconductor (China) Ltd, GEF",N,Y,Y
14930,MO997,Stepper,MA321,Canon,FPA-1550M2,"g-Line Stepper , PHOTO(Stepper), g-Line Steppe...",Stepper,411290,"1994, 1989, 1993","AVAIL, NOT_AVAIL, UNSPEC","The Innes Group, Sumitomo Mitsui Finance and L...","ROHM Tsukuba Co.,Ltd, KEC Corporation, AMI Sem...",N,Y,Y
14931,MO998,Stepper,MA321,Canon,FPA-1550M3-W,"STEPPER Main Body/Fi, PHOTO G-LINE STEPPER, PH...","RETICLE SIZE：φ125ｍｍ, STEPPER Main Body/Fi, A/E","901121M3W, 910325M3, 003349M3, 906183, 104384M...","1989, 1990, 1991.0","AVAIL, NOT_AVAIL, UNSPEC","Renesas Electronics Corporation, KEC Corporati...","Hangzhou E-Giant Semiconductor Tech Co., LTD, ...",N,Y,Y


In [19]:
result.to_csv("../../data/processed_model.csv", index=False)
result=pd.read_csv("../../data/processed_model.csv")

In [21]:
result.isna().sum()

modelID                0
modelSTDName           0
MakerID                0
modelAlias             0
category               0
makerAlias             0
makerSTDName           2
modelAlias_split       0
makerAlias_split       0
Process_nm          5043
Configuration       9832
dtype: int64

In [10]:
result.dropna(subset=["MODEL_NAME"], inplace=True)

In [11]:
result.isna().sum()

MODEL_ID                     0
Category_Name                0
MAKER_ID                     0
MAKER_NAME                 533
MODEL_NAME                   0
Process_NM              161989
Configuration            35875
SERIAL_NO               228867
VINTAGE                 242769
Tracker_availability         0
Seller                    9649
Owner                    12798
P&C                         57
STD_MODEL_YN                 0
STD_MAKER_YN                 0
dtype: int64

In [12]:
result.to_csv("../../data/processed_tracker_info.csv", index=False)